In [1]:
import numpy as np
import pickle as pi

# Matricies

This notebook uses many images from the excellent [A Visual Intro to NumPy and Data Representation](https://jalammar.github.io/visual-numpy/) from [Jay Alammar](https://jalammar.github.io/).

In the first notebook ([vector.ipynb](https://github.com/ADGEfficiency/teaching-monolith/blob/master/numpy/1.vector.ipynb)) we dealt with vectors (one dimensional). 

Now we deal with **Matricies** - arrays with two dimensions.

$\textbf{A}_{2, 2} = \begin{bmatrix}A_{1, 1} & A_{1, 2} \\ A_{2, 1} & A_{2, 2}\end{bmatrix}$

- two dimensional
- uppercase, bold $\textbf{A}_{m, n}$
- $A_{1, 1}$ = first element
- area
- tabular data

## Reshaping

Now that we have multiple dimensions, we need to start considering shape.

We can see the shape using `.shape`

In [7]:
data = np.arange(16)
data    

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [4]:
data.shape

(16,)

And the number of elements using `.size`

In [5]:
data.size

16

The **shape** of a matrix becomes more than just an indication of the length.  We can change the shape using reshape:

In [6]:
data.reshape(4,4)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

A very useful tool when reshaping is using `-1` - this is a free dimension that will be set to match the size of the data
- this is often set to the batch / number of samples dimension

In [8]:
data.reshape(2, -1)

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15]])

In [9]:
data.reshape(-1, 4)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

We can use `.reshape` to flatten

In [10]:
data.reshape(-1)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

We can also use `.flatten`

In [11]:
data = np.arange(16)

data.flatten() # returns new array
c[0] = 100
data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

And finally `.ravel`

In [12]:
data = np.arange(16)

data.ravel() # returns view of array
c[0] = 100
data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

Looking at the difference between ravel returning a view (not actual copy, just view of the original object)

`.flatten` always returns a copy - `.ravel` doesn't (if it can)

Closely related to a reshape is the **transpose**, which flips the array along the diagonal:

<img src="assets/trans.png" alt="" width="300"/>

In [14]:
np.arange(0, 6)

array([0, 1, 2, 3, 4, 5])

In [15]:
np.arange(6).reshape(3, -1)

array([[0, 1],
       [2, 3],
       [4, 5]])

In [16]:
# transpose with T
np.arange(6).reshape(3, -1).T

array([[0, 2, 4],
       [1, 3, 5]])

Reshape is (usually) computationally **cheap** - to understand why we need to know a little about how a `np.array` is laid out in memory

## The `np.array` in memory

- the data is stored in a single block
- the shape is stored as a tuple

Why is storing in a single block (known as a contiguous layout) a good thing?
- to access the next value an the array 
- we just move to the next memory address
- length = defined by the data type

> ... storing data in a contiguous block of memory ensures that the architecture of modern CPUs is used optimally, in terms of memory access patterns, CPU cache, and vectorized instructions - [iPython coobook](https://ipython-books.github.io/45-understanding-the-internals-of-numpy-to-avoid-unnecessary-array-copying/)

Changing the shape only means changing the tuple 
- the layout of the data in memory is not changed

The operations that will change the memory layout are ones that change the order of the data - for example a transpose:

In [17]:
data = np.arange(10000000).reshape(5, -1)
res = %timeit -qo data.reshape((1, -1))
'{:.8f} seconds'.format(res.average)

'0.00000039 seconds'

In [18]:
data = np.arange(10000000).reshape(5, -1)
res = %timeit -qo data.T.reshape((1, -1))
'{:.8f} seconds'.format(res.average)

'0.03221484 seconds'

## Two dimensional indexing

<img src="assets/idx2.png" alt="" width="500"/>

In [19]:
data = np.random.rand(2, 3)
data

array([[0.94728341, 0.22681411, 0.30238865],
       [0.28509744, 0.89903854, 0.02680071]])

We specify both dimensions using a familiar `[]` syntax

`:` = entire dimension

In [24]:
data[1:,1:]

array([[0.89903854, 0.02680071]])

`-1` = last element

In [25]:
# last column
data[:, -1]

array([0.30238865, 0.02680071])

### Two dimension aggregation

<img src="assets/agg-2d.png" alt="" width="900"/>

Now that we are working in two dimensions, we have more flexibility in how we aggregate
- we can specify the axis (i.e. the dimension) along which we aggregate

In [26]:
data

array([[0.94728341, 0.22681411, 0.30238865],
       [0.28509744, 0.89903854, 0.02680071]])

In [27]:
np.mean(data)

0.4479038088305554

In [29]:
np.mean(data, axis=0)

array([0.61619042, 0.56292632, 0.16459468])

In [30]:
np.mean(data, axis=1)

array([0.49216205, 0.40364556])

By default `numpy` will remove the dimension you are aggregating over:

In [31]:
data

array([[0.94728341, 0.22681411, 0.30238865],
       [0.28509744, 0.89903854, 0.02680071]])

In [32]:
# rows
np.mean(data, axis=1).shape

(2,)

You can choose to keep this dimension using a `keepdims` argument:

In [33]:
np.mean(data, axis=1, keepdims=True).shape

(2, 1)

## Practical

Aggregate by variance `np.var` 
- over the rows
- over the columns
- over all data

In [37]:
data

array([[0.94728341, 0.22681411, 0.30238865],
       [0.28509744, 0.89903854, 0.02680071]])

In [34]:
np.var(data, axis=0)

array([0.10962256, 0.11297142, 0.01898718])

In [35]:
np.var(data, axis=1)

array([0.10451964, 0.13382663])

In [36]:
np.var(data)

0.12113192935756867

## Two dimensional broadcasting

The general rule with broadcasting - dimensions are compatible when
- they are equal
- or when one of them is 1

<img src="assets/broad-2d.png" alt="" width="500"/>

In [40]:
data = np.arange(1, 7).reshape(3, 2)
data

array([[1, 2],
       [3, 4],
       [5, 6]])

In [43]:
data + np.array([0, 1, 1]).reshape(3, 1)

array([[1, 2],
       [4, 5],
       [6, 7]])

In [44]:
data + 1

array([[2, 3],
       [4, 5],
       [6, 7]])

## Matrix arithmetic

Can make arrays from nested lists:

In [46]:
np.array([[1, 2], [3, 4]])

array([[1, 2],
       [3, 4]])

In [47]:
np.array([[1, 2], [3, 4]]).shape

(2, 2)

We can add matricies of the same shape as expected:

<img src="assets/add-matrix.png" alt="" width="300"/>

In [49]:
np.ones_like(data)

array([[1, 1],
       [1, 1],
       [1, 1]])

In [50]:
np.zeros_like(data)

array([[0, 0],
       [0, 0],
       [0, 0]])

In [ ]:
data + np.ones_like(data)

## Matrix multiplication

This kind of matrix multiplication will often **change the shape** of the array
- this is what happens in neural networks

<img src="assets/dot1.png" alt="" width="900"/>

This operation can be visualized:

<img src="assets/dot2.png" alt="" width="900"/>

In [51]:
data = np.array([1, 2, 3])
powers_of_ten = np.array([10**n for n in range(6)]).reshape(3,2)
powers_of_ten

array([[     1,     10],
       [   100,   1000],
       [ 10000, 100000]])

This is done in numpy using either `np.dot()`:

In [52]:
np.dot(data, powers_of_ten)

array([ 30201, 302010])

Or calling the `.dot()` method on the array itself:

In [53]:
data.dot(powers_of_ten)

array([ 30201, 302010])

## Making arrays from nested lists

In [56]:
data = np.array([[1, 2], [3, 4]])
data

array([[1, 2],
       [3, 4]])

## Making arrays from shape tuples

The argument to these functions is a tuple

### `zeros`, `ones`, `full`

In [57]:
np.zeros((2, 3))

array([[0., 0., 0.],
       [0., 0., 0.]])

In [58]:
np.ones((2,4))

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [60]:
np.full((2, 4), 5)

array([[5, 5, 5, 5],
       [5, 5, 5, 5]])

### `zeros_like`, `ones_like`, `full_like`

Similar to counterparts above, except their shape is defined by another array:

In [62]:
parent = np.arange(10).reshape(2, 5)
parent

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [64]:
np.zeros_like(parent)

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [66]:
np.ones_like(parent)

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])

In [65]:
np.full_like(parent, 3)

array([[3, 3, 3, 3, 3],
       [3, 3, 3, 3, 3]])

### `empty`

Similar to `zeros`, except the array is filled with garbage from RAM 
- this is a bit quicker than `zeros`

In [69]:
d = np.empty(4)
for e in range(4):
    d[e] = e
d

array([0., 1., 2., 3.])

In [71]:
d = np.empty(4)
d


array([0.00000000e+000, 2.78138476e-309, 5.43472210e-322, 0.00000000e+000])

### `eye`

Identity matrix :

In [72]:
np.eye(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

The linear algebra verision of a 1

In [74]:
d = np.arange(4).reshape(2, 2)
d

array([[0, 1],
       [2, 3]])

*** dot product times the identity matrix is like multiplying by 1

In [75]:
np.dot(np.eye(2), d)

array([[0., 1.],
       [2., 3.]])

## Matrix Practice

1. Write a function (using numpy) to sort a diven array of shape 2 along the first axis (rows), second axis (column), and on a flattened array.

example: 

In [54]:
# Expected Output:
# Original array:
np.array([[10, 40],
          [30, 20]])
# Sort the array along the first axis:
np.array([[10, 20],
          [30, 40]])
# Sort the array along the last axis:
np.array([[10, 40],
          [20, 30]])
# Sort the flattened array:
np.array([10, 20, 30, 40])

array([10, 20, 30, 40])

In [86]:
array_1 = np.random.rand(4).reshape(2,2)
array_1

array([[0.47834607, 0.7465403 ],
       [0.68560333, 0.83635874]])

In [91]:
array_1.flatten

<function ndarray.flatten>

In [93]:
# Asis answer
def play_array(array):
    print(array[1,:])
    print(array_1[:,1])
    print(array_1.flatten())

play_array(array_1)

[0.68560333 0.83635874]
[0.7465403  0.83635874]
[0.47834607 0.7465403  0.68560333 0.83635874]


In [ ]:
# Answer Teacher

2. Write a function to get the indicies of the sorted elements of a given array

Expected Output:

Original array:


`[1023 5202 6230 1671 1682 5241 4532]`


Indices of the sorted elements of a given array:


`[0 3 4 6 1 5 2]`

In [155]:
# Asis Answer 1
array_2 = [1023, 5202, 6230, 1671, 1682, 5241, 4532]

def sort_by_index(array):
    sorted = np.argsort(array)
    print(sorted)
    
sort_by_index(array_2)


[0 3 4 6 1 5 2]


In [150]:
# Asis Answer 2
array_2 = [1023, 5202, 6230, 1671, 1682, 5241, 4532]
?np.argsort()


Object `np.argsort()` not found.


In [ ]:
# Other answers
def func(data):
    sorted_data = np.zeros_like(data)
    m = max(data) * 2
    for i in range(len(data)):
        sorted_data[i] = data.argmin()
        data[data.argmin()] = m
    return sorted_data

######

idx_list = []
for i in range(0, len(old_array)):
    idx_list.append(list(np.where(old_array == new_array[i]))[0][0])

3. Write a function to sort a specified number of elements from the beginning of a given array

Sample output:


Original array:


`[0.39536213 0.11779404 0.32612381 0.16327394 0.98837963 0.25510787 0.01398678 0.15188239 0.12057667 0.67278699]`


Sorted first 5 elements:

`[0.01398678 0.11779404 0.12057667 0.15188239 0.16327394 0.25510787 0.39536213 0.98837963 0.32612381 0.67278699]`

In [158]:
# Answer Asis

array_3 = "0.39536213 0.11779404 0.32612381 0.16327394 0.98837963 0.25510787 0.01398678 0.15188239 0.12057667 0.67278699".split()
array_3 = [float(i) for i in array_3]

print(np.partition(array_3, -5))

""" def sort_array_elements(array):
    print(np.sort(array_3))

print(sort_array_elements(array_3)) """


[0.11779404 0.12057667 0.01398678 0.15188239 0.16327394 0.25510787
 0.32612381 0.39536213 0.67278699 0.98837963]


' def sort_array_elements(array):\n    print(np.sort(array_3))\n\nprint(sort_array_elements(array_3)) '

In [ ]:
# Other answers
def select_sort():
    i = my_array.ravel()
    i[:5] = np.sort(my_array[:5])
    
    print(my_array)
    return my_array

#######

def sort_first(n, arr):
    arr1, arr2 = arr[:n], arr[n:]
    arr1 = np.sort(arr1)
    out = np.append(arr1, arr2)
    return out
sort_first(5, arr)


